In [1]:
%cd ..

/workspaces/Chatchat_AIMeng/src


In [2]:
%pwd

'/workspaces/Chatchat_AIMeng/src'

In [3]:
from datasets import load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import os

/workspaces/Chatchat_AIMeng/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [4]:
train_tokenized = load_from_disk('data/processed/train_tokenized')
val_tokenized = load_from_disk('data/processed/val_tokenized')
test_tokenized = load_from_disk('data/processed/test_tokenized')

In [5]:
train_tokenized

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4548
})

# Load Model

In [6]:
%cd src

[Errno 2] No such file or directory: 'src'
/workspaces/Chatchat_AIMeng/src


/workspaces/Chatchat_AIMeng/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [7]:
from utils.util_funcs import clear_gpu_memory

In [8]:
## Downloading Model 
from utils.constants import *
from utils.util_funcs import *


In [10]:
FINE_TUNED_MODEL_PATH, BASE_MODEL_PATH

('models/fine_tuned_t5', 'models/google/flan-t5-base')

In [14]:

original_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, torch_dtype=torch.float32)
## Saving Model
original_model.save_pretrained(BASE_MODEL_PATH)

## Freeing GPU Memory
del original_model
clear_gpu_memory()

In [22]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         126G   43G   77G  36% /
tmpfs            64M     0   64M   0% /dev
shm              64M  8.0K   64M   1% /dev/shm
/dev/root        29G   27G  2.1G  94% /vscode
/dev/loop4      126G   43G   77G  36% /workspaces
/dev/sda1       724G  1.2M  687G   1% /tmp
tmpfs            56G   12K   56G   1% /proc/driver/nvidia
tmpfs            23G  1.3M   23G   1% /run/nvidia-persistenced/socket
devtmpfs         56G     0   56G   0% /dev/nvidia0


In [11]:
loaded_original_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_PATH, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
clear_gpu_memory()

# Fine_tune

In [12]:

training_args = TrainingArguments(
    output_dir=FINE_TUNED_MODEL_PATH,
    save_total_limit=SAVE_TOTAL_LIMIT,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    evaluation_strategy=EVALUATION_STRATEGY,
)

In [13]:
trainer = Trainer(
    model=loaded_original_model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.185600,0.167391
2,0.134900,0.149535
3,0.100000,0.147326
4,0.065400,0.151656
5,0.043800,0.167526


TrainOutput(global_step=5685, training_loss=0.12946400353021545, metrics={'train_runtime': 2643.308, 'train_samples_per_second': 8.603, 'train_steps_per_second': 2.151, 'total_flos': 1.557138219466752e+16, 'train_loss': 0.12946400353021545, 'epoch': 5.0})

In [15]:
## Saving the Model
SAVE_MODEL_PATH = os.path.join(MODEL_DIR, 'save')
trainer.model.save_pretrained(SAVE_MODEL_PATH)
tokenizer.save_pretrained(SAVE_MODEL_PATH)

('models/save/tokenizer_config.json',
 'models/save/special_tokens_map.json',
 'models/save/tokenizer.json')

# Clear GPU

In [17]:
del loaded_original_model


NameError: name 'loaded_original_model' is not defined

In [18]:
clear_gpu_memory()